<a href="https://colab.research.google.com/github/Srinu027/AI_assitant_chabot/blob/main/AIChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install transformers

In [ ]:
pip install phonenumbers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.1 MB/s eta 0:00:00


In [ ]:
pip install streamlit transformers phonenumbers

In [ ]:
%%capture
from pyngrok import ngrok
ngrok.set_auth_token("2uUpKvP2WlcvRvTxsh7zfpRl8eg_3M8EkB8DhEhFoq7b6Mp3b")

In [ ]:
!pip install streamlit transformers torch pyngrok -q
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -q ngrok.zip
!./ngrok authtoken 2uRV7oPxP8i53t3gt7gviMPjgYY_6Bvz1deHZ7nwaTQpCusJ5

replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
%%writefile app.py
import streamlit as st
import json
import os
import logging
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import phonenumbers


logging.basicConfig(filename='app.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


@st.cache_resource
def load_model(attempt=0, max_attempts=3):
    try:
        with st.spinner("Loading model, please wait..."):
            tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
            model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
        logging.info("Model loaded successfully")
        st.success("Model loaded successfully!")
        return tokenizer, model
    except Exception as e:
        logging.error(f"Failed to load model: {e}")
        if attempt < max_attempts - 1:
            st.warning(f"Retrying to load model ({attempt + 1}/{max_attempts})...")
            return load_model(attempt + 1, max_attempts)
        else:
            st.error(f"Failed to load model after {max_attempts} attempts: {e}. Using fallback mode.")
            return None, None

tokenizer, model = load_model()
if tokenizer is None or model is None:
    st.warning("Running in fallback mode without AI-generated questions.")


fields = ["name", "email", "phone", "experience", "positions", "location", "tech_stack"]
field_prompts = {
    "name": "Please provide your full name (e.g., John Doe).",
    "email": "What’s your email address? (e.g., user@example.com)",
    "phone": "What’s your phone number? (e.g., 1234567890 or +12025550123)",
    "experience": "How many years of software development experience do you have? (e.g., 5 or 2.5)",
    "positions": "Which position(s) are you applying for? (e.g., Software Engineer)",
    "location": "Where are you currently located? (e.g., New York)",
    "tech_stack": "What’s your tech stack? List languages, frameworks, etc., separated by commas (e.g., Python, Django, SQL)"
}
ending_keywords = {"exit", "quit", "bye", "end"}


def validate_name(name):
    return bool(re.match(r"^[a-zA-ZÀ-ÿ\s'-]+$", name.strip()))

def validate_email(email):
    return bool(re.match(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$", email.strip()))

def validate_phone(phone):
    try:
        parsed = phonenumbers.parse(phone, "US")
        return phonenumbers.is_valid_number(parsed)
    except phonenumbers.phonenumberutil.NumberParseException:
        return False

def validate_experience(exp):
    try:
        exp_float = float(exp.strip())
        return 0 <= exp_float <= 50
    except ValueError:
        return False

def validate_non_empty(input_str):
    return bool(input_str.strip())

validation_functions = {
    "name": validate_name,
    "email": validate_email,
    "phone": validate_phone,
    "experience": validate_experience,
    "positions": validate_non_empty,
    "location": validate_non_empty,
    "tech_stack": validate_non_empty
}


def initialize_session_state():
    defaults = {
        "conversation_history": [],
        "collected_info": {},
        "technical_questions": [],
        "candidate_answers": [],
        "retry_count": 0,
        "current_field_index": 0,
        "current_question_index": 0,
        "phase": "collecting_info",
        "conversation_ended": False
    }
    for key, value in defaults.items():
        if key not in st.session_state:
            st.session_state[key] = value

initialize_session_state()

@st.cache_data
def generate_technical_questions(tech_stack):
    try:
        if tokenizer is None or model is None:
            raise Exception("Model not loaded, using defaults.")
        tech_list = [t.strip() for t in tech_stack.split(',') if t.strip()]
        tech_str = ", ".join(tech_list) if tech_list else "your tech stack"
        prompt = (
            f"Generate five technical interview questions for a developer proficient in {tech_str}. "
            f"Cover programming concepts, frameworks, best practices, and problem-solving. "
            f"Each question should be concise and relevant to the technologies mentioned. "
            f"Format as:\nQ1: [Question]\nQ2: [Question]\n...\nQ5: [Question]"
        )
        inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True).replace(prompt, "").strip()
        questions = []
        for line in generated_text.split('\n'):
            match = re.match(r'Q\d+:\s*(.+)', line.strip())
            if match and match.group(1).strip():
                questions.append(match.group(1).strip())
        default_questions = [
            f"What’s your experience with {tech_str}?",
            f"Describe a project you built using {tech_str}.",
            f"How do you optimize code in {tech_str}?",
            f"What challenges have you faced with {tech_str}?",
            f"Explain a key concept in {tech_str}."
        ]
        while len(questions) < 5:
            questions.append(default_questions[len(questions) % len(default_questions)])
        logging.info(f"Generated questions for tech stack: {tech_str}")
        return questions[:5]
    except Exception as e:
        logging.warning(f"Failed to generate questions: {e}. Using defaults.")
        tech_str = tech_stack if tech_stack.strip() else "your tech stack"
        return [
            f"What’s your experience with {tech_str}?",
            f"Describe a project you built using {tech_str}.",
            f"How do you optimize code in {tech_str}?",
            f"What challenges have you faced with {tech_str}?",
            f"Explain a key concept in {tech_str}."
        ]


def store_candidate_info():
    candidate_data = {
        "collected_info": st.session_state.collected_info,
        "technical_questions": st.session_state.technical_questions,
        "answers": st.session_state.candidate_answers,
        "timestamp": datetime.now().isoformat()
    }
    if not (st.session_state.collected_info or st.session_state.candidate_answers):
        logging.warning("No meaningful data to save.")
        return
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"data/candidate_{timestamp}.json"
    try:
        os.makedirs("data", exist_ok=True)
        with open(filename, "w") as f:
            json.dump(candidate_data, f, indent=2)
        logging.info(f"Data saved to {filename}")
        st.success(f"Data saved to {filename}!")
    except PermissionError as e:
        logging.error(f"Permission denied: {e}")
        st.error(f"Permission denied when saving data: {e}. Check folder permissions.")
    except Exception as e:
        logging.error(f"Failed to save data: {e}")
        st.error(f"Failed to save data: {e}. Here’s what was submitted:")
        st.json(candidate_data)

st.title("TalentScout Hiring Assistant")
st.write("Interact with the chatbot to provide your details and answer technical questions.")

if st.button("Reset Conversation"):
    for key in st.session_state.keys():
        del st.session_state[key]
    initialize_session_state()
    st.rerun()

if not st.session_state.conversation_history:
    initial_message = (
        "Hello! I’m TalentScout’s Hiring Assistant. I’ll guide you through the initial screening by "
        "collecting your details and asking tech-related questions. Type 'exit' to end anytime. "
        "Let’s start with your full name."
    )
    st.session_state.conversation_history.append({"role": "assistant", "content": initial_message})

for message in st.session_state.conversation_history[-10:]:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if not st.session_state.conversation_ended:
    user_input = st.chat_input("Your response:")
    if user_input:
        input_lower = user_input.lower().strip()
        st.session_state.conversation_history.append({"role": "user", "content": user_input})
        logging.info(f"User input: {user_input}")

        if input_lower in ending_keywords:
            st.session_state.conversation_ended = True
            st.session_state.conversation_history.append({"role": "assistant", "content":
                "Thank you! We’ll review your details and get back to you soon."})
            store_candidate_info()
            st.rerun()

        elif st.session_state.phase == "collecting_info":
            if st.session_state.current_field_index < len(fields):
                field = fields[st.session_state.current_field_index]
                validator = validation_functions[field]
                if validator(user_input):
                    value = float(user_input) if field == "experience" else user_input.strip()
                    st.session_state.collected_info[field] = value
                    st.session_state.current_field_index += 1
                    st.session_state.retry_count = 0
                    if st.session_state.current_field_index < len(fields):
                        next_field = fields[st.session_state.current_field_index]
                        st.session_state.conversation_history.append({"role": "assistant", "content": field_prompts[next_field]})
                    else:
                        st.session_state.phase = "confirming_info"
                        info_display = "\n".join([f"{k}: {v}" for k, v in st.session_state.collected_info.items()])
                        st.session_state.conversation_history.append({"role": "assistant", "content":
                            f"Please review your information:\n{info_display}\nIs this correct? (yes/no)"})
                else:
                    if st.session_state.retry_count < 3:
                        st.session_state.retry_count += 1
                        error_msgs = {
                            "name": "Please enter a valid name (letters, spaces, hyphens, apostrophes, and accents only).",
                            "email": "Please enter a valid email (e.g., user@example.com).",
                            "phone": "Please enter a valid phone number (e.g., 1234567890 or +12025550123).",
                            "experience": "Please enter a number between 0 and 50 (e.g., 5 or 2.5).",
                            "positions": "Please specify at least one position.",
                            "location": "Please provide your location.",
                            "tech_stack": "Please list your tech stack."
                        }
                        st.session_state.conversation_history.append({"role": "assistant", "content":
                            f"{error_msgs[field]} {field_prompts[field]}"})
                    else:
                        st.session_state.conversation_ended = True
                        st.session_state.conversation_history.append({"role": "assistant", "content":
                            "Too many invalid attempts. Ending conversation."})
                        store_candidate_info()
                st.rerun()

        elif st.session_state.phase == "confirming_info":
            if input_lower == "yes":
                st.session_state.phase = "asking_questions"
                with st.spinner("Generating technical questions..."):
                    st.session_state.technical_questions = generate_technical_questions(st.session_state.collected_info["tech_stack"])
                store_candidate_info()
                name = st.session_state.collected_info["name"]
                st.session_state.conversation_history.append({"role": "assistant", "content":
                    f"Thanks, {name}! Now, I’ll ask 5 technical questions. Here’s the first: {st.session_state.technical_questions[0]}"})
            elif input_lower == "no":
                st.session_state.conversation_history.append({"role": "assistant", "content":
                    f"Which field would you like to edit? ({', '.join(fields)} or 'none')"})
                st.session_state.phase = "editing_info"
            else:
                st.session_state.conversation_history.append({"role": "assistant", "content": "Please answer 'yes' or 'no'."})
            st.rerun()

        elif st.session_state.phase == "editing_info":
            if input_lower in fields:
                st.session_state.current_field_index = fields.index(input_lower)
                st.session_state.phase = "collecting_info"
                st.session_state.retry_count = 0
                st.session_state.conversation_history.append({"role": "assistant", "content": field_prompts[input_lower]})
            elif input_lower == "none":
                st.session_state.phase = "asking_questions"
                with st.spinner("Generating technical questions..."):
                    st.session_state.technical_questions = generate_technical_questions(st.session_state.collected_info["tech_stack"])
                store_candidate_info()
                name = st.session_state.collected_info["name"]
                st.session_state.conversation_history.append({"role": "assistant", "content":
                    f"Thanks, {name}! Now, I’ll ask 5 technical questions. Here’s the first: {st.session_state.technical_questions[0]}"})
            else:
                st.session_state.conversation_history.append({"role": "assistant", "content":
                    f"Invalid input. Please enter a field name ({', '.join(fields)}) or 'none'."})
            st.rerun()

        elif st.session_state.phase == "asking_questions":
            if not user_input.strip():
                if st.session_state.retry_count < 3:
                    st.session_state.retry_count += 1
                    st.session_state.conversation_history.append({"role": "assistant", "content":
                        f"Please provide an answer. {st.session_state.technical_questions[st.session_state.current_question_index]}"})
                else:
                    st.session_state.conversation_ended = True
                    st.session_state.conversation_history.append({"role": "assistant", "content":
                        "Too many empty responses. Ending conversation."})
                    store_candidate_info()
            else:
                st.session_state.candidate_answers.append(user_input.strip())
                st.session_state.current_question_index += 1
                st.session_state.retry_count = 0
                if st.session_state.current_question_index < len(st.session_state.technical_questions):
                    next_q = st.session_state.technical_questions[st.session_state.current_question_index]
                    st.session_state.conversation_history.append({"role": "assistant", "content": next_q})
                else:
                    name = st.session_state.collected_info["name"]
                    st.session_state.conversation_ended = True
                    st.session_state.conversation_history.append({"role": "assistant", "content":
                        f"Great job, {name}! That’s all. We’ll review your responses soon."})
                    store_candidate_info()
            st.rerun()

        else:
            st.session_state.conversation_history.append({"role": "assistant", "content":
                "Something went wrong. Please answer the current question or provide the requested info."})
            st.rerun()
else:
    st.write("The conversation has ended. Thank you for participating!")

Overwriting app.py


In [ ]:
%%writefile requirements.txt
streamlit
transformers
phonenumbers

Writing requirements.txt


In [ ]:
%%writefile .gitignore
# Ignore Streamlit secrets
.streamlit/secrets.toml
# Ignore cached data
data/
# Ignore Python cache
__pycache__/
# Ignore logs
*.log

Writing .gitignore


In [ ]:
%%writefile README.md
1. Project Overview:
Brief description of the chatbot:
The TalentScout Hiring Assistant is an AI-driven chatbot built with Streamlit and Hugging Face's `gpt-neo-1.3B` model. It automates candidate screening by collecting information (name, tech stack, etc.) and generating technical questions tailored to their skills.

2. Installation Instructions:
## Installation
3. Clone the repository:
   ```bash
   git clone https://github.com/your-username/talentscout-hiring-assistant.git
   cd talentscout-hiring-assistant
   ```
4. Install the required packages:
   ```bash
   pip install -r requirements.txt
   ```
5. Run the Streamlit app:
   ```bash
   streamlit run app.py

**3. Usage Guide**
_How to interact with the chatbot:_
```markdown
## Usage
1. Open the app in your browser.
2. Provide your details when prompted (name, email, tech stack, etc.).
3. Answer 5 technical questions generated based on your tech stack.
4. Type "exit" to end the conversation.

4. Technical Details:
- **Libraries**: Streamlit, Hugging Face Transformers, `phonenumbers` for validation.
- **Model**: `EleutherAI/gpt-neo-1.3B` for question generation.
- **Data Handling**: Anonymized candidate data stored in JSON files.

5. Prompt Design:
- **Information Gathering**: Structured prompts with validation (e.g., regex for emails).
- **Question Generation**: Dynamic prompts based on tech stack (e.g., "Generate 5 questions for Python/Django").

6. Challenges & Solutions:
- **Model Loading Failures**: Added retry logic and fallback questions.
- **Data Privacy**: Anonymized data storage and excluded sensitive files via `.gitignore`.



Writing README.md


In [ ]:
# Kill processes using port 4040 (ngrok's default)
!kill $(lsof -t -i:4040)  # For Linux-based Colab environments
# OR
!pkill ngrok

In [1]:
%%capture
!streamlit run app.py &>/dev/null&

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2025-03-18 15:51:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 13.248.244.96, 75.2.60.68, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.28M  67.7MB/s    in 0.2s    

2025-03-18 15:52:00 (67.7 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [ ]:
# Step 1: Install required packages
!pip install -q streamlit pyngrok rich

# Step 2: Create necessary directories
!mkdir -p .streamlit data

# Step 3: Configure Streamlit and ngrok
import os
from pyngrok import ngrok, conf

# Set Streamlit config
os.environ['STREAMLIT_REPORT_STREAM_ERRORS'] = 'false'

# REPLACE WITH YOUR NEW AUTHTOKEN FROM NGROK DASHBOARD
NGROK_AUTH_TOKEN = "2uUpKvP2WlcvRvTxsh7zfpRl8eg_3M8EkB8DhEhFoq7b6Mp3b"

# Configure ngrok
conf.get_default().auth_token = NGROK_AUTH_TOKEN
conf.get_default().monitor_thread = False

# Step 4: Start Streamlit app in background
from multiprocessing import Process
import time

def run_streamlit():
    os.system("streamlit run app.py --server.headless true")

streamlit_process = Process(target=run_streamlit)
streamlit_process.start()

# Wait for Streamlit to initialize
time.sleep(10)

# Step 5: Create ngrok tunnel
try:
    tunnel = ngrok.connect(8501)  # Ensure port matches Streamlit's default (8501)
    ngrok_url = tunnel.public_url
    print(f"\nYour app is accessible at: {ngrok_url}")
except Exception as e:
    print(f"Error creating tunnel: {str(e)}")
    print("Ensure your authtoken is valid and ngrok is properly configured.")


Your app is accessible at: https://68ff-35-190-171-163.ngrok-free.app


In [2]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [3]:
!git init
!git config --global user.email "srinu.manchala2003@gmail.com"
!git config --global user.name "Srinu027"

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [4]:
!git config --global --list

user.email=srinu.manchala2003@gmail.com
user.name=Srinu027


In [5]:
!git clone https://github.com/Srinu027/AI_assitant_chabot.git

Cloning into 'AI_assitant_chabot'...


In [10]:
!ls -l

total 24
drwxr-xr-x 3 root root  4096 Mar 19 17:32 AI_assitant_chabot
-rw-r--r-- 1 root root 15620 Mar 19 17:33 app.py
drwxr-xr-x 1 root root  4096 Mar 17 13:32 sample_data


In [26]:
!mv app.py AI_assitant_chabot/
%cd AI_assitant_chabot

mv: cannot move 'app.py' to 'AI_assitant_chabot/': Not a directory
[Errno 2] No such file or directory: 'AI_assitant_chabot'
/content/streamlit-app/AI_assitant_chabot


In [25]:
!git add .
!git commit -m "Initial commit: Add Streamlit app"
!git push origin master

[master (root-commit) 72d3604] Initial commit: Add Streamlit app
 1 file changed, 319 insertions(+)
 create mode 100644 streamlit-app/AI_assitant_chabot/app.py
fatal: 'origin' does not appear to be a git repository
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
